In [ ]:
import argparse
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataloader
from torchvision import datasets, transforms
from src.models import DSNet

In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    model.train()

In [ ]:
def train(args, model, mean_teacher, device, train_loader, test_loader, optimizer, epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        output = model(data)

        with torch.no_grad():
            mean_t_output = mean_teacher(data)

        const_loss = F.mse_loss(output, mean_t_output)

        weight = 0.2
        loss = F.nll_loss(output, target) + weight*const_loss
        loss.backward()
        optimizer.step()

        alpha = 0.95
        for mean_param, param in zip(mean_teacher.parameters(), model.parameters()):
            mean_param.data.mul_(alpha).add_(1 - alpha, param.data)

        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
            test(model, device, test_loader)
            test(mean_teacher, device, test_loader)

In [43]:
epochs = 10
lr = 0.01
momentum = 0.5
random_seed = 42

In [44]:
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    dev = "cuda"
else:
    dev = "cpu"
device = torch.device(dev)
print(f"DEVICD : {dev}")

DEVICD : cuda


In [47]:
def CIFAR10_dataset_on(args, kwargs):
    transform_CIFAR10 = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    trainset_CIFAR10 = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_CIFAR10)
    testset_CIFAR10 = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_CIFAR10)

    train_loader_CIFAR10 = DataLoader(trainset_CIFAR10, batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader_CIFAR10 = DataLoader(testset_CIFAR10, batch_size=args.batch_size, shuffle=True, **kwargs)
    return train_loader_CIFAR10, test_loader_CIFAR10

In [ ]:
def main():
    parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Example')
    parser.add_argument('--batch-size', type=int, default=4, metavar='N',
                        help='input batch size for training (default: 4)')
    parser.add_argument('--test-batch-size', type=int, default=100, metavar='N',
                        help='input batch size for testing (default: 100)')
    parser.add_argument('--epochs', type=int, default=10, metavar='N',
                        help='number of epochs to train (default: 10)')
    parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                        help='learning rate (default: 0.01)')
    parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                        help='SGD momentum (default: 0.5)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')

    parser.add_argument('--save-model', action='store_true', default=False,
                        help='For Saving the current Model')
    args = parser.parse_args()
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}
    train_loader, test_loader = CIFAR10_dataset_on(args, kwargs)

    model = DSNet(device, "checkpoints/student-cifar10.pth")
    mean_teacher = DSNet(device, "checkpoints/mean_teacher-cifar10.pth")
    model = model.to(device)
    mean_teacher = mean_teacher.to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, mean_teacher, device, train_loader, test_loader, optimizer, epoch)

    if (args.save_model):
        torch.save(model.state_dict(), "mnist_cnn.pt")